In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_DNN_FC
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train = np.load(path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_test = np.load(path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_valid = np.load(path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')


In [3]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train, X_valid, X_test)

In [4]:
dnn_normalized = Genome3D_DNN_FC(num_features=1920, use_deep_CNN=False)
validation_data = (X_valid_normalized, y_valid)
dnn_normalized.train(X_train_normalized, y_train, validation_data)

Training model...
Train on 6710 samples, validate on 1796 samples
Epoch 1/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.4648Epoch 0: validation loss: 0.805
Balanced Accuracy: 53.17%	auROC: 0.566	 auPRC: 0.575
Positve Accuracy: 0.89%	 Negative Accuracy: 89.76%
Recall at 5% | 10% | 25% | 50% FDR: 16.6% | 1.0% | 1.0% | 99.9%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 2s - loss: 1.4624 - val_loss: 0.8049
Epoch 2/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.2011Epoch 1: validation loss: 0.732
Balanced Accuracy: 54.79%	auROC: 0.618	 auPRC: 0.621
Positve Accuracy: 0.89%	 Negative Accuracy: 85.63%
Recall at 5% | 10% | 25% | 50% FDR: 23.9% | 0.9% | 16.6% | 99.9%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 1s - loss: 1.2004 - val_loss: 0.7316
Epoch 3/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.0431Epoch 2: validation loss: 0.680
Balanced Accuracy:

In [5]:
print(dnn_normalized.test(X_test_normalized, y_test))
print(dnn_normalized.test(X_train_normalized, y_train))
print(dnn_normalized.test(X_valid_normalized, y_valid))

Balanced Accuracy: 58.30%	auROC: 0.635	 auPRC: 0.657
Positve Accuracy: 1.21%	 Negative Accuracy: 63.36%
Recall at 5% | 10% | 25% | 50% FDR: 53.2% | 2.4% | 26.3% | 99.8%	Num Positives: 494	 Num Negatives: 494

Balanced Accuracy: 99.94%	auROC: 1.000	 auPRC: 1.000
Positve Accuracy: 99.97%	 Negative Accuracy: 99.91%
Recall at 5% | 10% | 25% | 50% FDR: 100.0% | 100.0% | 100.0% | 100.0%	Num Positives: 3355	 Num Negatives: 3355

Balanced Accuracy: 69.27%	auROC: 0.759	 auPRC: 0.755
Positve Accuracy: 0.00%	 Negative Accuracy: 76.61%
Recall at 5% | 10% | 25% | 50% FDR: 61.9% | 22.3% | 50.1% | 100.0%	Num Positives: 898	 Num Negatives: 898



In [6]:
#save model and weights
dnn_normalized.save(contacts+'_dnn_FC_seq_features_edges_'+'thres_'+thres, '/users/mtaranov/genome3D/')